In [ ]:
import os
os.chdir("..")
import actions
import funcs_database
import funcs_supabase
import requests


settings = {}
project_id = 27
final_video_name = 'project-27'


words_list = funcs_supabase.select_data("video_creator_projects", "id", project_id)[0]["words_list"]
print(words_list)

In [ ]:
def make_folder(final_video_name):
    base_path = "composed_videos"
    new_folder_path = os.path.join(base_path, final_video_name)

    # Create the new folder if it doesn't exist
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        print(f"'{new_folder_path}' has been created!")
    else:
        print(f"'{new_folder_path}' already exists!")

In [ ]:

def download_file_from_url(url, save_path):
    """
    Download a file from the provided URL and save it to the specified path.

    Parameters:
    - url (str): The URL of the file to download.
    - save_path (str): The local path where the file should be saved.
    """
    
    # Ensure the directory structure exists
    directory = os.path.dirname(save_path)
    os.makedirs(directory, exist_ok=True)
    
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"File downloaded and saved to {save_path}")

# Example usage:
# download_file_from_url("https://example.com/file.zip", "local_folder/file.zip")


In [ ]:
from moviepy.editor import AudioFileClip

# Download speech and subs
project_data = funcs_supabase.select_data(table='video_creator_projects', column='id', value=project_id)
print(project_data)

filepath = f"composed_videos/{final_video_name}/subs.srt"
subs = project_data[0]['subtitles_file']
download_file_from_url(subs, filepath)

filepath = f"composed_videos/{final_video_name}/speech.mp3"
speech = project_data[0]['speech_audio']
download_file_from_url(speech, filepath)
speech_clip = AudioFileClip(filepath)



last_word_end_time = words_list[-1]['end']
first_word_time = words_list[0]['start']
print(f"first word start time {first_word_time}")
print(f"last word end time {last_word_end_time}")

speech_audio = "composed_videos/project17-2/speech.mp3"
audio = AudioFileClip(speech_audio)
audio_duration = audio.duration # this will print the duration of the audio in seconds
print(f"audio duration {audio_duration}")
audio.close()




In [ ]:
import os
from urllib.parse import urlparse, unquote

def get_file_extension(url):
    # Unquote the URL to handle any encoded characters
    url = unquote(url)
    # Parse the URL into its components
    parsed_url = urlparse(url)
    # Get the path component of the URL and then extract the file extension
    return os.path.splitext(parsed_url.path)[1][1:]

def download_media_for_list(scenes_data_list):
    downloaded_files_list = []
    for sc in scenes_data_list:
        # print(i)
        # print(sc)
        media_url = sc.get('media_url', None)
        media_type = sc.get('media_type', None)
        media_url_remote = sc.get('media_url_remote', None)
        scene_id = sc.get('id', None)


        # sometimes if url is missing but remote url is present
        if media_url is None and media_url_remote is not None:
            media_url = media_url_remote

        url_extension = get_file_extension(media_url)
        if url_extension == '':
            if media_type == 'image':
                url_extension = 'png'
            elif media_type == 'video':
                url_extension = 'mp4'
            else:
                url_extension = 'png'


                
        print(url_extension)

        filename = f"composed_videos/{final_video_name}/{scene_id}.{url_extension}"
        print(filename)

        download_file_from_url(media_url, filename)

        downloaded_files_list.append(filename)

    return downloaded_files_list




    # print(media_url)
    # print(media_type)
    # print(media_url_remote)


In [ ]:
import zipfile
import os

def zip_folder(folder_path, output_filename):
    """
    Zips the contents of a folder into a .zip file.
    
    Parameters:
    - folder_path: Path to the folder you want to zip.
    - output_filename: Name of the output .zip file.
    """
    
    # Create a ZipFile Object
    with zipfile.ZipFile(output_filename, 'w') as zipf:
        # Walk through the folder and add all files to the ZipFile
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                # Create complete filepath of file in directory
                file_path = os.path.join(foldername, filename)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, folder_path))



# folder = f"composed_videos/{final_video_name}"
# zip_folder(folder, final_video_name + '.zip')


In [ ]:
from moviepy.editor import VideoFileClip

def trim_clip(clip, start_trim, end_trim):
    """
    Trim a video clip to fit a new length between start_trim and end_trim.
    
    :param clip: VideoFileClip instance representing the video to trim.
    :param start_trim: Start trim point in seconds.
    :param end_trim: End trim point in seconds.
    
    :return: Adjusted video clip.
    """
    
    current_duration = clip.duration
    target_duration = end_trim - start_trim

    print(f"Current clip length: {current_duration}")
    print(f"Target length: {target_duration}")

    if current_duration == target_duration:
        print("Clip is already the target duration.")
        return clip
    elif current_duration > target_duration:
        print("Clip is longer than target duration. Trimming...")
        return clip.subclip(start_trim, end_trim)
    else:
        print("Clip is shorter than target duration. Slowing down to fit...")
        speed_factor = current_duration / target_duration
        return clip.speedx(speed_factor)

# Example usage
# clip = VideoFileClip("composed_videos/final_video123/0.mp4")
# new_clip = trim_clip(clip, start_trim=4, end_trim=8)
# new_clip.write_videofile("output_video.mp4")




from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop

def crop_clip(video_clip, width=1600, height=900):
    """
    Crop a video to a specific size, maintaining its center.
    
    :param video_clip: VideoFileClip instance of the video.
    :param width: Desired width of the cropped video.
    :param height: Desired height of the cropped video.
    
    :return: Cropped (and possibly resized) VideoFileClip.
    """
    
    # If clip's width and height match the desired dimensions, return the clip
    if video_clip.w == width and video_clip.h == height:
        print("Video is already the desired size")
        return video_clip
    
    current_aspect_ratio = video_clip.w / video_clip.h
    target_aspect_ratio = width / height

    # Calculate the cropping dimensions based on the aspect ratios
    if current_aspect_ratio > target_aspect_ratio:
        crop_width = video_clip.h * target_aspect_ratio
        crop_height = video_clip.h
    else:
        crop_width = video_clip.w
        crop_height = video_clip.w / target_aspect_ratio

    # Crop the video clip
    cropped_clip = crop(video_clip, width=crop_width, height=crop_height, 
                        x_center=video_clip.w/2, y_center=video_clip.h/2)

    # Resize the cropped video to the desired dimensions
    resized_clip = cropped_clip.resize(width=width, height=height)

    return resized_clip


# Usage
# clip = VideoFileClip("composed_videos/final_video123/0.mp4")
# new_clip = crop_clip(clip, width=1600, height=900)
# new_clip.write_videofile("output_video_2.mp4")


In [ ]:
def get_scenes_data_list(project_id):
    scenes_list = actions.get_active_scenes(project_id)
    
    scenes_data_list = []
    for scene in scenes_list:
        scene_id = scene['id']
        scene_data = funcs_database.get_scene_and_clip_by_id(scene_id)
        # print(scene_data)
        scenes_data_list.append(scene_data)

    return scenes_data_list


scenes_data_list = get_scenes_data_list(project_id)
downloaded_files_list = download_media_for_list(scenes_data_list)

if len(downloaded_files_list) == len(scenes_data_list):
    print('All files downloaded successfully')

# print(downloaded_files_list)
# print(scenes_data_list)







In [ ]:
for scene in scenes_data_list:
    print(scene["id"])
    print(scene["timings"])

In [ ]:
total_length_scenes = 0
for item in scenes_data_list:
    print(item["timings"])
    # total_length_scenes += item["timings"["length"]]
    total_length_scenes = total_length_scenes + item["timings"]["length"]
print(total_length_scenes)



from moviepy.editor import AudioFileClip

def get_duration_of_mp3(file_path):
    with AudioFileClip(file_path) as audio:
        return audio.duration

filepath = f"composed_videos/{final_video_name}/speech.mp3"

duration_in_seconds = get_duration_of_mp3(filepath)
print(f"Duration of the MP3 file: {duration_in_seconds:.2f} seconds")


In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, ImageClip  
import moviepy.editor as mpy
from moviepy.video.fx.all import crop


import math


initial_clip_list = []
for file in downloaded_files_list:
    print(file)

    scene_id = file.split('/')[-1].split('.')[0]
    print(scene_id)

    scene_data = funcs_database.get_scene_and_clip_by_id(scene_id)
    print(scene_data)

    timings = scene_data['timings']
    length = timings['length']


    if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        clip = VideoFileClip(file)
        initial_clip_list.append(clip)


    elif file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        # Set the duration for how long you want each image to display
        clip = ImageClip(file, duration=length)
        initial_clip_list.append(clip)

    else:
        print(f"Unsupported format: {file}")



    # clip = VideoFileClip(file)
    # initial_clip_list.append(clip)

print(initial_clip_list)






In [ ]:
proceessed_clip_list = []

for i, clip in enumerate(initial_clip_list):

    scene_data = scenes_data_list[i]
    timings = scene_data['timings']
    length = timings['length']

    # print(f"scene length: {length}")
    
    trimmed_clip = trim_clip(clip, start_trim=0, end_trim=length)
    resized_clip = crop_clip(trimmed_clip, width=900, height=900)

    proceessed_clip_list.append(resized_clip)






In [ ]:
for c in proceessed_clip_list:
    print(c)


In [ ]:
from moviepy.editor import concatenate_audioclips, AudioClip
import numpy as np

def combine_video_audio(video_clip, audio_clip, audio_silence):
    # silence_array = np.zeros((int(audio_silence * audio_clip.fps),)).astype("float32")
    silence_array = np.zeros((int(audio_silence * audio_clip.fps * audio_clip.nchannels),)).astype("float32")



    silence = AudioClip(lambda t: silence_array, duration=audio_silence).set_fps(audio_clip.fps)



    print(silence.fps, silence.nchannels, silence.duration)
    print(speech_clip.fps, speech_clip.nchannels, speech_clip.duration)


    combined_audio = concatenate_audioclips([silence, audio_clip])

    combined_audio.write_audiofile("test_combined_audio.mp3")


        
    return result

In [ ]:
from moviepy.editor import *
import numpy as np

def add_gap_to_audio(audio_clip, beginning_gap, output_path):
    # Load audio
    # audio_clip = AudioFileClip(audio_path)
    
    # Create a silent audio clip of the given duration (gap)
    silent_clip = AudioClip(lambda t: [0, 0], duration=beginning_gap).set_fps(audio_clip.fps)
    
    # Concatenate silent clip with the original audio
    new_audio = concatenate_audioclips([silent_clip, audio_clip])
    
    # Write the audio with gap to the output file
    new_audio.write_audiofile(output_path)

# Example of using the function
# add_gap_to_audio("input_audio.mp3", 5, "output_with_gap.mp3")


In [ ]:
from moviepy.editor import *

def join_audio_video(audio, video, output_path):
    """
    Join given audio and video files and save to output_path.

    :param audio_path: path to the audio file
    :param video_path: path to the video file
    :param output_path: path to save the output video file
    """
    
    # Load audio and video clips
    # audio = AudioFileClip(audio_path)
    # video = VideoFileClip(video_path)
    
    # Set audio to video
    video_with_audio = video.set_audio(audio)
    
    # Write the result to file
    video_with_audio.write_videofile(output_path, audio_codec='aac')

    return output_path

# Example usage:
# join_audio_video("path_to_audio.mp3", "path_to_video.mp4", "output_video.mp4")


In [ ]:
final_clip = concatenate_videoclips(proceessed_clip_list)
final_clip = final_clip.without_audio()
print(final_clip.duration)


speech_filepath = f"composed_videos/{final_video_name}/speech.mp3"
speech_clip = AudioFileClip(speech_filepath)
print(speech_clip.duration)

# silence_gap = final_clip.duration - speech_clip.duration
# silence_gap = round(silence_gap, 3)

# new_audio_file = "output_with_gap.mp3"
# add_gap_to_audio(speech_clip, silence_gap, new_audio_file)
# output_audio_clip = AudioFileClip(new_audio_file)
# print(output_audio_clip.duration)


join_audio_video(speech_clip, final_clip, "output_video1.mp4")



# final_clip = combine_video_audio(final_clip, audio_clip=speech_clip, audio_silence=silence_gap)

# # final_clip.write_videofile("yolo7.mp4")
# final_clip.write_videofile("output.mp4", codec="libx264", audio_codec="aac")
